# Process anitSMASH outputs

In [1]:
from os import listdir, rename, path
import pandas as pd
from Bio import SeqIO
from bs4 import BeautifulSoup
import csv
import re
from collections import defaultdict

### Clean up folder names from antiSMASH raw output

In [ ]:
antismash_results = "../antismash_results"

for assembly in listdir(antismash_results):
    if assembly == ".DS_Store": # MacOS thing
        continue
    if assembly.endswith("_assembly"):
        new_name = assembly.replace("_assembly", "")
        current_path = path.join(antismash_results, assembly)
        new_path = path.join(antismash_results, new_name)

        if path.exists(new_path):
            continue
        else:
            rename(current_path, new_path)

### Modify filenames to avoid duplicate names when running BiG-SCAPE

In [ ]:
antismash_results = "../antismash_results"

for assembly in listdir(antismash_results):
    assembly_path = f"{antismash_results}/{assembly}"
    if assembly in [".DS_Store"]: # MacOS thing
        continue

    for file in listdir(assembly_path):
        if ".region" in file:
            new_name = f"{assembly}_{file}"
            current_path = path.join(assembly_path, file)
            new_path = path.join(assembly_path, new_name)
            
            if path.exists(new_path):
                continue
            else:
                rename(current_path, new_path)

### Extract all BGC annotations from antiSMASH region GenBank files

In [5]:
def extract_bgc_annos(assembly_path: str, assembly: str):
    """ Extract annotations from each BGC predicted by antiSMASH """

    all_annos = []  # Master list to store annotations from all GenBank files

    for file in listdir(assembly_path):
        if ".region" in file:
            file_path = f"{assembly_path}/{file}"
            
            with open(file_path, "r") as gbk:
                records = SeqIO.parse(gbk, "genbank")

                for record in records:
                    for feature in record.features:
                        if feature.type == "protocluster":
                            tmp_dict = {}
                            tmp_dict["Position"] = file.split(".gbk")[0]

                            number = feature.qualifiers.get("protocluster_number").pop()
                            category = feature.qualifiers.get("category").pop()
                            product = feature.qualifiers.get("product").pop()

                            tmp_dict["Protocluster"] = number
                            tmp_dict["Category"] = category
                            tmp_dict["Product"] = product
                            
                            references = []
                            for q in feature.qualifiers:
                                if q.endswith("product_classes"):
                                    references = feature.qualifiers.get(q, [])  # Get reference(s) if found
                            
                            if not references:
                                references = [product]  # Default to product if no reference found

                            for ref in references:
                                new_entry = tmp_dict.copy()
                                new_entry["Reference"] = ref
                                new_entry["Assembly"] = assembly
                                all_annos.append(new_entry)

    if all_annos:
        df = pd.DataFrame(all_annos)

        column_order = ["Assembly", "Position", "Protocluster", "Category", "Product", "Reference"]
        df = df[column_order]

        # Save individual assembly BGC annotations
        output_path = f"{assembly_path}/{assembly}_bgc_annotations.csv"
        df.to_csv(output_path, index=False)

        return df
    
    return None

def process_all_folders(antismash_results_path: str):
    """Loops through all subdirectories in 'antismash_results' and processes them."""
    all_dfs = []  # List to collect all individual DataFrames

    for assembly in listdir(antismash_results_path):
        if assembly == ".DS_Store":  # Skip system files (MacOS thing)
            continue
        
        assembly_path = f"{antismash_results_path}/{assembly}"
        df = extract_bgc_annos(assembly_path=assembly_path, assembly=assembly)

        if df is not None:
            all_dfs.append(df)

    # Merge all DataFrames into one
    if all_dfs:
        merged_df = pd.concat(all_dfs, ignore_index=True)
        merged_df.to_csv(f"{antismash_results_path}/all_bgc_annotations_from_region_gbk.csv", index=False)
        return merged_df

    return "No BGC annotations found in any assembly!"

# Run the processing
antismash_results = "../antismash_results"
merged_annotations = process_all_folders(antismash_results)

### Extract BGC annotations from the HTML files

In [ ]:
def get_bgcs_from_html(assembly_path: str, assembly: str):
    """ Extract more specific BGC annotations from antiSMASH index.html output file """

    data = {"SEQID": [], "Product": []}

    html_path = f"{assembly_path}/index.html"
    with open (html_path) as fp:
        soup = BeautifulSoup(fp, 'html.parser')

    unique_links = set()

    for td in soup.find_all("td"):
        a_tags = td.find_all("a", class_="external-link")

        if len(a_tags) == 1 and a_tags[0].get("target") == "_blank":
            text = a_tags[0].text.strip()
            if text not in unique_links:
                unique_links.add(text)
    
    if unique_links: 
        for i in unique_links:
            data["Product"].append(i)
        data["SEQID"] = [assembly] * len(unique_links)

    else:
        data["Product"].append("Unknown bioactive compound")
        data["SEQID"].append(assembly)
    
    return data

def get_all_html_bgcs(results_folder_path):
    master = []
    for assembly in listdir(results_folder_path):
        if assembly != ".DS_Store": # MacOS thing
            result = get_bgcs_from_html(assembly_path = f"{results_folder_path}/{assembly}", assembly=assembly)
            master.append(result)

    master_df = pd.DataFrame(master)
    master_df = master_df.explode(["SEQID", "Product"])

    pd.DataFrame.to_csv(master_df, "../antismash_results/all_bgc_annotations_from_html.csv", index=False)
    
    return master_df

antismash_results = "../antismash_results"

get_all_html_bgcs(antismash_results)

### Extract BGC annotations from JSON file

In [ ]:
from os import listdir
import json
import pandas as pd

def get_bgcs_from_json(assembly, assembly_path):
    df = {
        "SEQID": [],
        "Contig": [],
        "Region": [],
        "Category": [],
        "Number of hits": [],
        "Most similar known cluster": [],
        "Similarity": []
    }

    for file in listdir(assembly_path):
        if file.endswith("assembly.json"):
            file_path = f"{assembly_path}/{file}"
            with open(file_path) as f:
                data = json.load(f)
                records = data.get("records")

                for i in range(len(records)):
                    record = records[i]
                    contig = record["id"]
                    module = record["modules"]

                    for contig_region in record["features"]:
                        if contig_region["type"] == "region":
                            qualifier = contig_region["qualifiers"]
                            region = qualifier["region_number"]
                            p = qualifier["product"]
                            df["SEQID"].append(assembly)    # Get SEQID
                            df["Contig"].append(contig)     # Get BGC contig number
                            df["Region"].append(region[0])  # Get BGC region number
                            df["Category"].append(p[0])     # Get BGC category

                    for modkey in module.keys():

                        # Getting ClusterBlast hits (all BGC-like regions):
                        if modkey == "antismash.modules.clusterblast":
                            line = module[modkey]

                            c = line["knowncluster"]
                            c_results = c["results"]

                            for region_result in c_results:
                                ranks = region_result["ranking"]
                                n_hits = region_result["total_hits"]
                                df["Number of hits"].append(n_hits)

                                bgc_list = []
                                similarity_list = []

                                for l in ranks:
                                    for hits_dict in l:
                                        if "description" in hits_dict:
                                            bgc = hits_dict["description"]
                                            bgc_list.append(bgc)

                                        if "similarity" in hits_dict:
                                            similarity = hits_dict["similarity"]/100
                                            similarity_list.append(similarity)

                                refs_dict = {bgc_list[i]: similarity_list[i] for i in range(len(bgc_list))}
                                if len(refs_dict):
                                    best_bgc = max(refs_dict, key=refs_dict.get)
                                    best_bgc_similarity = refs_dict[best_bgc]

                                    df["Most similar known cluster"].append(best_bgc)
                                    df["Similarity"].append(best_bgc_similarity)
                                
                                if not len(refs_dict):
                                    df["Most similar known cluster"].append(None)
                                    df["Similarity"].append(0)

    return pd.DataFrame(df)

def get_all_json_bgcs(antismash_path):
    master_df = []

    for assembly in listdir(antismash_path):
        if assembly != ".DS_Store": # MacOS thing
            assembly_path = f"{antismash_path}/{assembly}"
            df = get_bgcs_from_json(assembly, assembly_path)
            master_df.append(df)

    master_df = pd.concat(master_df, ignore_index=True)

    # Replace NAs for missing most similar known clusters with category
    master_df["Most similar known cluster"] = master_df["Most similar known cluster"].fillna(master_df["Category"])

    return pd.DataFrame.to_csv(master_df, "../antismash_results/all_bgc_annotations_from_json.csv", index=False)

get_all_json_bgcs("../antismash_results/")

""


# bakta results

### Extract gene annotations from embl file

In [ ]:
def wga_to_csv(input_file, output_file, file_format="embl"):
    """Converts a whole-genome annotation in .gbk or .embl format to a CSV with gene/product info per contig and totals."""

    with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
        writer = csv.writer(outfile)
        headers = ["SEQID", "Gene", "Product"]
        writer.writerow(headers)

        global_totals = defaultdict(int)

        for record in SeqIO.parse(infile, file_format):
            seqid = record.id

            for feature in record.features:
                if feature.type == "CDS":
                    gene = feature.qualifiers.get("gene", [""])[0]
                    product = feature.qualifiers.get("product", [""])[0]

                    writer.writerow([seqid, gene, product])

                    # Optional: keep totals if needed
                    global_totals["total_genes"] += 1
                    if gene:
                        global_totals[gene] += 1

# Run on bakta output from all assemblies
for assembly in listdir("../bakta_results/"):
    embl_file = f"../bakta_results/{assembly}/{assembly}_assembly.embl"
    if embl_file == "../bakta_results/.DS_Store/.DS_Store.embl": # MacOS thing
        continue
    wga_to_csv(embl_file, f"../bakta_results/{assembly}/{assembly}_bakta.csv", file_format="embl")

# Merge all bakta annotations into a single csv file
master = []
for assembly in listdir("../bakta_results"):
    assembly_path = f"../bakta_results/{assembly}"
    if assembly == ".DS_Store":
        continue
    
    for file in listdir(assembly_path):
        if file.endswith("bakta.csv"):
            bakta_csv_path = f"{assembly_path}/{file}"
            bakta_df = pd.read_csv(bakta_csv_path)
            bakta_df["SEQID"] = assembly
            master.extend(bakta_df[["SEQID", "Gene", "Product"]].to_dict(orient="records"))

master_df = pd.DataFrame(master)

pd.DataFrame.to_csv(master_df, "../bakta_results/all_bakta_annos.csv", index=False)